#### Imports

In [3]:
import sys
import nltk.sentiment
import pandas as pd
import nltk

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


#### Read the dataset

In [4]:
data = pd.read_csv("output.csv", keep_default_na=False)

#### Define the text preprocessing functions

In [7]:
def pre_process(text):

    #First tokenize the text to continue
    tokens = nltk.tokenize.word_tokenize(text.lower())

    #remove stop words:
    stopword_list = nltk.corpus.stopwords.words('english')

    filtered_tokens = [token for token in tokens if token not in stopword_list]

    lemmatizer = nltk.stem.WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]


    processed_text = ' '.join(lemmatized_tokens)

    return processed_text



def sentiment_analysis_nltk(processed_text):

    analyzer = nltk.sentiment.vader.SentimentIntensityAnalyzer()

    score = analyzer.polarity_scores(processed_text)

    return score['compound']

def sentiment_analysis_huggingface(text):
    dictionary = sentiment_pipeline(text)[0]
    if dictionary['label'] == 'NEGATIVE':
        result = -dictionary['score']
    else:
        result = dictionary['score']
    return result

#### Run the functions on the dataframe

In [8]:
data['Processed text'] = data['Content'].apply(pre_process)
data['Sentiment_nltk'] = data['Processed text'].apply(sentiment_analysis_nltk)

data['Sentiment_huggingface'] = data['Content'].apply(sentiment_analysis_huggingface)

<class 'list'>


TypeError: list indices must be integers or slices, not str

#### Save the results in a new csv file

In [5]:

data.to_csv('test.csv')